In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Set display options for better viewing in Jupyter
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', 1000)

In [2]:
alarms_df = pd.read_csv('./given_data/datasets/Alarms.csv')
anomalies_df = pd.read_csv('./given_data/datasets/Anomalies.csv')
sessions_df = pd.read_csv('./given_data/datasets/Sessions.csv')
stations_df = pd.read_csv('./given_data/datasets/Stations.csv')

In [3]:
# Remove columns with all null values
columns_to_drop = ['timezone_offset', 'status', 'time_stamp', 'mode']
stations_df_clean = stations_df.drop(columns=columns_to_drop)

# Verify the results
print(f"Original Stations shape: {stations_df.shape}")
print(f"Cleaned Stations shape: {stations_df_clean.shape}")

# Display the cleaned dataframe columns
print("\nCleaned Stations columns:")
print(stations_df_clean.columns.tolist())

stations_df = stations_df_clean

# Basic info on the cleaned dataframe
print("\nCleaned Stations info:")
print(stations_df.info())

Original Stations shape: (38, 23)
Cleaned Stations shape: (38, 19)

Cleaned Stations columns:
['station_id', 'org_id', 'station_group', 'model', 'activation_dt', 'address', 'manufacturer', 'station_name', 'description', 'port_no', 'reservable', 'level', 'connector', 'voltage', 'current', 'power', 'estimated_cost', 'location_lat', 'location_long']

Cleaned Stations info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   station_id      38 non-null     object 
 1   org_id          38 non-null     object 
 2   station_group   38 non-null     object 
 3   model           38 non-null     object 
 4   activation_dt   38 non-null     object 
 5   address         38 non-null     object 
 6   manufacturer    38 non-null     object 
 7   station_name    38 non-null     object 
 8   description     38 non-null     object 
 9   port_no         38 non-n

In [4]:
# Check for duplicate station_ids
station_id_duplicates = stations_df['station_id'].duplicated().sum()
if station_id_duplicates == 0:
    print("PASSED: All station_ids are unique in stations_df")
else:
    print(f"WARNING: Found {station_id_duplicates} duplicate station_ids")

In [5]:
# Check if all station_ids in sessions_df exist in stations_df
station_ids_in_stations = set(stations_df['station_id'])
station_ids_in_sessions = set(sessions_df['station_id'])
orphaned_station_ids = station_ids_in_sessions - station_ids_in_stations

if len(orphaned_station_ids) == 0:
    print("PASSED: All station_ids in sessions_df exist in stations_df")
else:
    print(f"WARNING: Found {len(orphaned_station_ids)} station_ids in sessions_df that don't exist in stations_df")
    print(list(orphaned_station_ids)[:5])  # Show first 5 as examples

['23ad8d0095', '956028aa4c', '00c4ca50ea', '162185ed79', '0dc8ed1cc1']


In [6]:
# Merge to analyze the relationship
merged_df = sessions_df.merge(stations_df[['station_id', 'port_no']],
                             on='station_id',
                             suffixes=('_session', '_station'))

# Check if session port numbers are within the valid range for each station
merged_df['valid_port'] = (merged_df['port_no_session'] >= 1) & (merged_df['port_no_session'] <= merged_df['port_no_station'])
invalid_ports = sum(~merged_df['valid_port'])

if invalid_ports == 0:
    print("PASSED: All port numbers in sessions are within valid range for their stations")
else:
    print(f"WARNING: Found {invalid_ports} sessions using invalid port numbers")

# Show distribution of ports per station
print("\nDistribution of port_no in stations_df:")
print(stations_df['port_no'].value_counts().sort_index())

# Show distribution of which ports are used in sessions
print("\nDistribution of port_no in sessions_df:")
print(sessions_df['port_no'].value_counts().sort_index())


Distribution of port_no in stations_df:
port_no
1    19
2    19
Name: count, dtype: int64

Distribution of port_no in sessions_df:
port_no
1    34578
2    31431
Name: count, dtype: int64


In [7]:
# Test 1: Check if (station_id, port_no) is unique
compound_key_duplicates = stations_df.duplicated(subset=['station_id', 'port_no']).sum()
print(f"Duplicated (station_id, port_no) pairs: {compound_key_duplicates}")

# Test 2: Analyze station_id counts
station_counts = stations_df['station_id'].value_counts()
print("\nDistribution of station_id occurrences:")
print(station_counts.value_counts())  # How many stations appear once, twice, etc.

# Test 3: Check port distribution per station
port_distribution = stations_df.groupby('station_id')['port_no'].apply(list)
print("\nSample of port numbers per station:")
print(port_distribution.head())

# Test 4: Verify all stations have expected port patterns
stations_with_ports_1_and_2 = sum([(set(ports) == {1, 2}) for ports in port_distribution])
print(f"\nStations with exactly ports 1 and 2: {stations_with_ports_1_and_2}")
print(f"Total unique stations: {len(port_distribution)}")

# Test 5: Analyze the missing station references
print("\nExamining orphaned sessions:")
orphaned_sessions = sessions_df[~sessions_df['station_id'].isin(stations_df['station_id'])]
print(f"Number of orphaned sessions: {len(orphaned_sessions)}")
print(f"Unique orphaned station IDs: {orphaned_sessions['station_id'].nunique()}")

# Test 6: Check port numbers in orphaned sessions
print("\nPort numbers used in orphaned sessions:")
print(orphaned_sessions['port_no'].value_counts())

Duplicated (station_id, port_no) pairs: 0

Distribution of station_id occurrences:
count
2    19
Name: count, dtype: int64

Sample of port numbers per station:
station_id
0342d90fcf    [1, 2]
047effd886    [1, 2]
0b22b7c266    [1, 2]
18173fd803    [1, 2]
1ff7182ea0    [1, 2]
Name: port_no, dtype: object

Stations with exactly ports 1 and 2: 19
Total unique stations: 19

Examining orphaned sessions:
Number of orphaned sessions: 36974
Unique orphaned station IDs: 13

Port numbers used in orphaned sessions:
port_no
1    19288
2    17686
Name: count, dtype: int64


In [8]:
# Identify orphaned sessions
orphaned_sessions = sessions_df[~sessions_df['station_id'].isin(stations_df['station_id'])]

# Get unique station_id and port_no combinations
orphaned_pairs = orphaned_sessions[['station_id', 'port_no']].drop_duplicates()

# Count sessions per combination
pair_counts = orphaned_sessions.groupby(['station_id', 'port_no']).size().reset_index(name='session_count')

# Sort by station_id and port_no for readability
pair_counts = pair_counts.sort_values(['station_id', 'port_no'])

print(f"Total unique (station_id, port_no) pairs: {len(pair_counts)}")
print("\nOrphaned station-port combinations with session counts:")
print(pair_counts)

Total unique (station_id, port_no) pairs: 26

Orphaned station-port combinations with session counts:
    station_id  port_no  session_count
0   00c4ca50ea        1             75
1   00c4ca50ea        2             69
2   01335ad401        1            134
3   01335ad401        2             77
4   0dc8ed1cc1        1           6557
..         ...      ...            ...
21  a9571abfa4        2            106
22  a96c3b900c        1             76
23  a96c3b900c        2             86
24  bba10a9daf        1           1716
25  bba10a9daf        2           1748

[26 rows x 3 columns]


In [9]:
# Get the orphaned sessions
orphaned_sessions = sessions_df[~sessions_df['station_id'].isin(stations_df['station_id'])]

# Group by station_id and check for addresses
station_addresses = orphaned_sessions.groupby('station_id')['address'].first().reset_index()

# Check if all orphaned stations have address information
missing_addresses = station_addresses['address'].isnull().sum()

print(f"Orphaned stations with address information: {len(station_addresses) - missing_addresses} out of {len(station_addresses)}")
print("\nStation IDs with their addresses:")
print(station_addresses)

# Check if each station has consistent address information
address_consistency = orphaned_sessions.groupby('station_id')['address'].nunique().reset_index()
address_consistency.columns = ['station_id', 'unique_address_count']
inconsistent_stations = address_consistency[address_consistency['unique_address_count'] > 1]

print(f"\nStations with multiple addresses: {len(inconsistent_stations)}")
if len(inconsistent_stations) > 0:
    print("Stations with inconsistent addresses:")
    print(inconsistent_stations['station_id'].tolist())

Orphaned stations with address information: 13 out of 13

Station IDs with their addresses:
    station_id                                            address
0   00c4ca50ea  8888 University Dr W, Floor P7000, Burnaby, Br...
1   01335ad401  8888 University Dr W, Burnaby, British Columbi...
2   0dc8ed1cc1  8915 Cornerstone Mews, Burnaby, British Columb...
3   162185ed79  8999 Nelson Way, Burnaby, British Columbia, V5...
4   18db03c780  8888 University Dr W, Burnaby, British Columbi...
5   23ad8d0095  8888 University  Drive, Burnaby, British Colum...
6   4ce863e612  8888 University Dr W, Burnaby, British Columbi...
7   615db15a48  13419 103 Ave Floor, Surrey, British Columbia,...
8   87adf1b6d0  13419 103 Ave, Floor P1, Surrey, British Colum...
9   956028aa4c  8888 University Dr W, Floor P7000, Burnaby, Br...
10  a9571abfa4  8888 University Dr W, Floor P7000, Burnaby, Br...
11  a96c3b900c  8888 University Dr W, Floor P7000, Burnaby, Br...
12  bba10a9daf  8999 Nelson Way, Burnaby, British 

In [10]:
# List of stations with inconsistent addresses
inconsistent_station_ids = ['0dc8ed1cc1', '162185ed79', '23ad8d0095', '615db15a48', '87adf1b6d0']

# For each inconsistent station, get all unique addresses
for station_id in inconsistent_station_ids:
    # Filter sessions for this station
    station_sessions = orphaned_sessions[orphaned_sessions['station_id'] == station_id]

    # Get unique addresses
    unique_addresses = station_sessions['address'].unique()

    print(f"\nStation ID: {station_id}")
    print(f"Number of unique addresses: {len(unique_addresses)}")
    print("Unique addresses:")
    for i, address in enumerate(unique_addresses, 1):
        print(f"{i}. {address}")


Station ID: 0dc8ed1cc1
Number of unique addresses: 2
Unique addresses:
1. 8915 Cornerstone Mews, Burnaby, British Columbia, V5A 4Y6, Canada
2. 8911 Cornerstone Mews, Burnaby, British Columbia, V5A 4Y6, Canada

Station ID: 162185ed79
Number of unique addresses: 2
Unique addresses:
1. 8999 Nelson Way, Burnaby, British Columbia, V5G 4N2, Canada
2. 8999 Nelson Way, Burnaby, British Columbia, V5A 4W9, Canada

Station ID: 23ad8d0095
Number of unique addresses: 2
Unique addresses:
1. 8888 University  Drive, Burnaby, British Columbia, V5A1S6, Canada
2. 8888 University Dr, Burnaby, British Columbia, V5A 1S6, Canada

Station ID: 615db15a48
Number of unique addresses: 2
Unique addresses:
1. 13419 103 Ave Floor, Surrey, British Columbia, V3T 1S6, Canada
2. 13419 103 Ave, Surrey, British Columbia, V3T 1S6, Canada

Station ID: 87adf1b6d0
Number of unique addresses: 2
Unique addresses:
1. 13419 103 Ave, Floor P1, Surrey, British Columbia, V3T 1R7, Canada
2. 13419 103 Ave, Surrey, British Columbia, V

In [14]:
# Create a dataframe for the missing stations
new_stations_data = []

# New station information
station_info = [
    {"station_id": "0dc8ed1cc1", "address": "8915 Cornerstone Mews, Burnaby, British Columbia, V5A 4Y6, Canada", "location_lat": 49.27806955715426, "location_long": -122.91203807404098},
    {"station_id": "162185ed79", "address": "8999 Nelson Way, Burnaby, British Columbia, V5G 4N2, Canada", "location_lat": 49.27380817107044, "location_long": -122.9122931912351},
    {"station_id": "23ad8d0095", "address": "8888 University Drive, Burnaby, British Columbia, V5A1S6, Canada", "location_lat": 49.2797337375215, "location_long": -122.92418284890641},
    {"station_id": "615db15a48", "address": "13419 103 Ave, Surrey, British Columbia, V3T 1S6, Canada", "location_lat": 49.190413065036694, "location_long": -122.85111038200404},
    {"station_id": "87adf1b6d0", "address": "13419 103 Ave, Floor P1, Surrey, British Columbia, V3T 1R7, Canada", "location_lat": 49.19048327828886, "location_long": -122.85091700288174},
    {"station_id": "00c4ca50ea", "address": "8888 University Dr W, Floor P7000, Burnaby, British Columbia, Canada", "location_lat": 49.27963567579469, "location_long": -122.92405411918756},
    {"station_id": "01335ad401", "address": "8888 University Dr W, Burnaby, British Columbia, Canada", "location_lat": 49.27963567579469, "location_long": -122.92405411918756},
    {"station_id": "18db03c780", "address": "8888 University Dr W, Burnaby, British Columbia, Canada", "location_lat": 49.27963567579469, "location_long": -122.92405411918756},
    {"station_id": "4ce863e612", "address": "8888 University Dr W, Burnaby, British Columbia, Canada", "location_lat": 49.27963567579469, "location_long": -122.92405411918756},
    {"station_id": "956028aa4c", "address": "8888 University Dr W, Floor P7000, Burnaby, British Columbia, Canada", "location_lat": 49.27963567579469, "location_long": -122.92405411918756},
    {"station_id": "a9571abfa4", "address": "8888 University Dr W, Floor P7000, Burnaby, British Columbia, Canada", "location_lat": 49.27963567579469, "location_long": -122.92405411918756},
    {"station_id": "a96c3b900c", "address": "8888 University Dr W, Floor P7000, Burnaby, British Columbia, Canada", "location_lat": 49.27963567579469, "location_long": -122.92405411918756},
    {"station_id": "bba10a9daf", "address": "8999 Nelson Way, Burnaby, British Columbia, V5A 4W9, Canada", "location_lat": 49.27380817107044, "location_long": -122.91220736054855}
]

# Create a DataFrame with just the columns we're filling
for station in station_info:
    for port in [1, 2]:
        row = {
            'station_id': station['station_id'],
            'port_no': port,
            'address': station['address'],
            'location_lat': station['location_lat'],
            'location_long': station['location_long'],
            'reservable': 0
        }
        new_stations_data.append(row)

# Create a dataframe with the new stations
new_stations_df = pd.DataFrame(new_stations_data)

# Get integer column names from original dataframe
int_columns = stations_df.select_dtypes(include=['int64']).columns.tolist()

# For other columns that need non-null values, use appropriate defaults
for col in stations_df.columns:
    if col not in new_stations_df.columns:
        if col in int_columns:
            # For integer columns, use 0 as default
            new_stations_df[col] = 0
        else:
            # For other columns, use empty string for object types
            if stations_df[col].dtype == 'object':
                new_stations_df[col] = ''

# Combine with the original stations dataframe
stations_df = pd.concat([stations_df, new_stations_df], ignore_index=True)

print(f"Combined stations count: {len(stations_df)}")

Combined stations count: 64


In [15]:
# Ensure date columns are in datetime format
if not pd.api.types.is_datetime64_dtype(sessions_df['start_dt']):
    sessions_df['start_dt'] = pd.to_datetime(sessions_df['start_dt'])

if not pd.api.types.is_datetime64_dtype(sessions_df['end_dt']):
    sessions_df['end_dt'] = pd.to_datetime(sessions_df['end_dt'])

# Get the min and max dates
min_date = sessions_df['start_dt'].min()
max_date = sessions_df['end_dt'].max()

# Calculate the duration of the dataset
date_range_days = (max_date - min_date).days

print(f"Dataset spans from {min_date.date()} to {max_date.date()}")
print(f"Total time period: {date_range_days} days ({date_range_days/30.44:.1f} months)")

# Count sessions by month to check distribution
sessions_df['year_month'] = sessions_df['start_dt'].dt.to_period('M')
monthly_counts = sessions_df['year_month'].value_counts().sort_index()

print("\nSessions per month:")
print(monthly_counts)

Dataset spans from 2019-11-06 to 2025-04-17
Total time period: 1989 days (65.3 months)

Sessions per month:
year_month
2019-11     479
2019-12     538
2020-01     622
2020-02     646
2020-03     627
           ... 
2024-12    1663
2025-01    2015
2025-02    1874
2025-03    2072
2025-04    1218
Freq: M, Name: count, Length: 66, dtype: int64
